In [12]:


import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import re

executable_path = r"C:/Users/Chris/Downloads/chromedriver_win32/chromedriver.exe"
driver = webdriver.Chrome(executable_path=executable_path)

url = 'https://www.loblaws.ca/store-locator?type=store&icta=pickup-details-modal'
driver.get(url)




C:\Users\Chris\AppData\Local\Temp\ipykernel_36624\1621092237.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=executable_path)


In [14]:


from selenium.webdriver.common.keys import Keys

# Find the location search input field
search_input = driver.find_element(By.ID, 'location-search__search__input')

# Clear the input field if it already has a value
search_input.clear()

# Enter the desired address or location
search_input.send_keys('Rideau Street')

# Press Enter to perform the search
search_input.send_keys(Keys.ENTER)

select_location_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.location-set-store__button')))
select_location_button.click()

# Wait for the button to be clickable
wait = WebDriverWait(driver, 10)
continue_shopping_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.fulfillment-location-confirmation__actions__button')))
continue_shopping_button.click()



In [15]:

url = 'https://www.loblaws.ca/food/fruits-vegetables/c/28000?navid=flyout-L2-fruits-vegetables'
driver.get(url)

grocery_list = []

while True:
    # Wait for the groceries to load
    wait = WebDriverWait(driver, 20)
    groceries = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'product-tracking')))
    
    for grocery in groceries:
        try:
            brand = grocery.find_element(By.CLASS_NAME, 'product-name__item--brand').text
        except:
            brand = "Unknown"

        try:
            product = grocery.find_element(By.CLASS_NAME, 'product-name__item--name').text
        except:
            product = "Unknown"

        try:
            price_text = grocery.find_element(By.CLASS_NAME, 'selling-price-list__item__price--now-price__value').text
            price = float(re.search(r'[\d.]+', price_text).group())
        except:
            price = 0.0

        grocery_dict = {'product brand': brand, "product name": product, "price": price, "store name": "RIDEAU STREET"}
        grocery_list.append(grocery_dict)
    
    try:
        # Scroll to the end of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Wait for the page to load
        
        # Find the "Load More" button
        load_more_button = driver.find_element(By.CSS_SELECTOR, '.primary-button.primary-button--load-more-button')
        if load_more_button.is_displayed():
            load_more_button.click()
        else:
            break
    except NoSuchElementException:
        break
    except Exception as e:
        print(str(e))
        break


In [16]:


driver.quit()


In [17]:


import pandas as pd

# Convert the grocery_list to a pandas DataFrame
df = pd.DataFrame(grocery_list)

# Print the DataFrame
print(df)



           product brand                     product name  price  \
0                Unknown                   Bananas, Bunch   1.75   
1                Unknown                 English Cucumber   1.29   
2                Unknown          Red Seedless Watermelon   7.99   
3                Unknown                      Green Onion   1.49   
4                Unknown                     Red Cherries   7.83   
...                  ...                              ...    ...   
4994            Marzetti      Veggie Dip, Southwest Ranch   5.99   
4995     Farmer's Market                       Red Onions   5.49   
4996  President's Choice  Salad Kit Chopped Sesame Wonton   6.99   
4997             Unknown            LoBok (Daikon Radish)   3.09   
4998             Unknown                     Red Potatoes   1.43   

         store name  
0     RIDEAU STREET  
1     RIDEAU STREET  
2     RIDEAU STREET  
3     RIDEAU STREET  
4     RIDEAU STREET  
...             ...  
4994  RIDEAU STREET  
4995  R

In [18]:
df.to_csv('ottawa_rideau_street.csv')